In [ ]:
import time
import os
import numpy as np
from pynq import allocate
from pynq import MMIO
from pynq import Overlay

In [ ]:
os.getcwd()
ol = Overlay("dma_loopback.bit")

In [ ]:
# base addr
DMA_GRABBER = 0x40000000
AXI_GPIO    = 0x41200000

# GPIO
dma_garbber_en     = 0x10
dma_garbber_size   = 0x18
fifo_remain_data   = 0x0000
grabber_pass_data  = 0x0008

In [ ]:
# 写寄存器
def write_reg(base_addr, offset, val):
    mmio = MMIO(base_addr, 64*1024)   # 创建一个名为mimo的MMIO对象,可用地址空间为64KB(与VIVADO设计中的地址分配相吻合)
    mmio.write(offset, val)           # 向地址为base_addr+offset的寄存器写入val


# 读寄存器
def read_reg(base_addr, offset):  
    mmio = MMIO(base_addr, 64*1024)
    read_val = mmio.read(offset)
    return read_val


# 写DMA
def dma_write(data):
    dma = ol.axi_dma_0
    data_in = allocate(shape=(len(data),), dtype=np.uint32)
    np.copyto(data_in, data)
    dma.sendchannel.transfer(data_in)
    dma.sendchannel.wait()
    data_in.close()


# 读DMA(size=0时,读出当前DMA中的全部数据)
def dma_read(size):
    dma = ol.axi_dma_0
    available_num = read_reg(AXI_GPIO, fifo_remain_data) + 2 # FIFO中的有效数据量
    target_size = available_num if size == 0 else size       # 本次DMA读取的目标数据量
    input_buffer = allocate(target_size, dtype=np.uint32)
    dma.recvchannel.transfer(input_buffer)
    write_reg(DMA_GRABBER,dma_garbber_en,0)
    write_reg(DMA_GRABBER,dma_garbber_size,target_size)
    write_reg(DMA_GRABBER,dma_garbber_en,1)
    dma.recvchannel.wait()
    data = input_buffer.copy()
    input_buffer.freebuffer()
    real_num = read_reg(AXI_GPIO, grabber_pass_data) # 本次DMA读取的实际数据量
    data = data[:real_num]
    if available_num > 2 :
        print(f"{real_num} data are read this time, and {available_num-real_num} data remain in the FIFO.")
    else :
        if real_num==0 or real_num==2  :
           print(f"{real_num} data are read this time, and 0 data remain in the FIFO.")
        else :
            if real_num < target_size :
                print(f"{real_num} data are read this time, and 0 data remain in the FIFO.")
            else :
                print(f"{real_num} data are read this time, and 0 or 1 data remain in the FIFO.")
    return(real_num,data)

In [ ]:
tx_data = np.arange(100, dtype=np.uint)
dma_write(tx_data)

In [ ]:
(rx_size, rx_data) = dma_read(0)
print(rx_data)